In [1]:
from src.extract import *
from src.transform import *
from FlightRadar24.api import FlightRadar24API

22/09/10 20:10:59 WARN Utils: Your hostname, sasadi resolves to a loopback address: 127.0.1.1; using 10.188.67.52 instead (on interface wlo1)
22/09/10 20:10:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/10 20:10:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/10 20:11:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
not_cleaned_data = '/home/sadi/Bureau/exaltKata/spark_pipeline/data/without_cleaning'
cleaned_data = '/home/sadi/Bureau/exaltKata/spark_pipeline/data/clean'
flights_path = os.path.join(not_cleaned_data, 'flights.json')
airports_path = os.path.join(cleaned_data, 'airports.json')
airlines_path = os.path.join(cleaned_data, 'airlines.json')

In [3]:
fr_api = FlightRadar24API()
flights = fr_api.get_flights()
#get flights
flight_details = get_flight_details(flights, fr_api)
save_original_file(flight_details, flights_path)

100%|██████████| 1500/1500 [17:01<00:00,  1.47it/s]


In [4]:
#get airports
airports = fr_api.get_airports()
save_original_file(airports, airports_path)

In [5]:
#get airlines
airlines = fr_api.get_airlines()
save_original_file(airlines, airlines_path)

### Transform flights DataFrame


In [3]:
flights_df = load_original_data(flights_path)

In [4]:
flights_clean_df = transform_flights_df(flights_df)

In [9]:
flights_clean_df.printSchema()

root
 |-- aircraft_age: string (nullable = true)
 |-- aircraft_code: string (nullable = true)
 |-- aircraft_country_id: string (nullable = true)
 |-- aircraft_images: string (nullable = true)
 |-- aircraft_model: string (nullable = true)
 |-- airline_iata: string (nullable = true)
 |-- airline_icao: string (nullable = true)
 |-- airline_name: string (nullable = true)
 |-- airline_short_name: string (nullable = true)
 |-- altitude: long (nullable = true)
 |-- callsign: string (nullable = true)
 |-- destination_airport_altitude: string (nullable = true)
 |-- destination_airport_baggage: string (nullable = true)
 |-- destination_airport_country_code: string (nullable = true)
 |-- destination_airport_country_name: string (nullable = true)
 |-- destination_airport_gate: string (nullable = true)
 |-- destination_airport_iata: string (nullable = true)
 |-- destination_airport_icao: string (nullable = true)
 |-- destination_airport_latitude: string (nullable = true)
 |-- destination_airport_lo

##### compagny most active flights in the world

In [6]:
flights_clean_df.select('airline_short_name', 'status_icon')\
    .filter(F.col("status_icon") == "green")\
    .groupBy(F.col('airline_short_name'), F.col('status_icon'))\
    .count()\
    .orderBy(F.col('count').desc())\
    .show(5)

+------------------+-----------+-----+
|airline_short_name|status_icon|count|
+------------------+-----------+-----+
|   Delta Air Lines|      green|   83|
|   United Airlines|      green|   74|
| American Airlines|      green|   74|
|          Emirates|      green|   35|
|        Air France|      green|   31|
+------------------+-----------+-----+
only showing top 5 rows



##### companies with most regional active flights

In [31]:

flights_clean_df.select('airline_short_name', 'status_icon',F.col('destination_airport_country_name').alias('airport_country'), 'sameCountry')\
    .filter((F.col('status_icon') == 'green') & F.col('sameCountry') == True)\
    .groupBy('airport_country', 'airline_short_name')\
    .count()\
    .orderBy(F.col('airport_country'))\
    .show()

+---------------+--------------------+-----+
|airport_country|  airline_short_name|count|
+---------------+--------------------+-----+
|      Australia|             Jetstar|    1|
|         Canada|             WestJet|    2|
|         Canada|          Air Canada|    5|
|         Russia|            NordStar|    1|
|         Russia|            Aeroflot|    1|
|         Russia|       Ural Airlines|    1|
|         Russia|         S7 Airlines|    1|
|         Russia|             Rossiya|    3|
|  United States|     Delta Air Lines|   18|
|  United States|           Atlas Air|    1|
|  United States|                 N/A|    1|
|  United States|               FedEx|   17|
|  United States|     Spirit Airlines|    4|
|  United States|Western Global Ai...|    1|
|  United States|     United Airlines|   13|
|  United States|Amerijet Internat...|    1|
|  United States|     JetBlue Airways|   14|
|  United States|  Southwest Airlines|    2|
|  United States|      Breeze Airways|    1|
|  United 